# Quran: Load Quran text (Arabic and English) into a file via public APIs

In [ ]:
import requests
import pandas as pd

def getSurah(x: int) -> dict:
    """Get Surah Arabic and English as dictionary"""
    if x < 1 or x > 114:
        return {
            "status": 400
        }
    df = []
    url = f"http://api.alquran.cloud/v1/surah/{x}/quran-uthmani"
    response = requests.get(url)
    translation = requests.get(f"http://api.alquran.cloud/v1/surah/{x}/en.asad")
    response.raise_for_status()
    translation.raise_for_status()
    ordered = response.json()
    trans = translation.json()
    data = ordered["data"]["ayahs"]
    tran = trans["data"]["ayahs"]
    if len(data) != len(tran):
        print("Error!")
    else:
        n = len(data)
        for i in range(n):
            df.append({
                "Arabic": data[i]["text"],
                "English": tran[i]["text"]
            })
    return df

def surahToDB(df: dict) -> pd.DataFrame:
    """Inputs getSurah(x) and outputs database"""
    db = pd.DataFrame(df)
    return db

def saveSurah(x: int):
    """Save Surah to local file as dataframe"""
    df = getSurah(x)
    pass

# To Anki (genanki)

In [ ]:
import requests

# Define the AnkiConnect API endpoint
ANKI_CONNECT_URL = "http://127.0.0.1:8765"

def add_flashcard(front, back, deck_name = "a"):
    """Adds a flashcard to Anki using AnkiConnect"""
    payload = {
        "action": "addNote",
        "version": 6,
        "params": {
            "note": {
                "deckName": deck_name,
                "modelName": "Basic",
                "fields": {
                    "Front": front,
                    "Back": back
                },
                "tags": ["python_generated"],
                "options": {
                    "allowDuplicate": False
                }
            }
        }
    }

    # Send the request to AnkiConnect
    response = requests.post(ANKI_CONNECT_URL, json=payload)
    result = response.json()

    if result.get("error"):
        print(f"Error: {result['error']}")
    else:
        print(f"Flashcard added to {deck_name} (Note ID: {result['result']})")


---

# Run

In [ ]:
def toAnki(mini: int, maxi: int, deck = "a"):
    for i in range(mini, maxi): # Includes both mini and maxi params
        print("Fetching Chapter ", i)
        surah = getSurah(i)
        n = len(surah)
        for j in range(0, n, 5):
            tmp_str = ""
            lim = min(j+5, len(surah))
            for verse in surah[j:lim]:
                tmp_str += verse["Arabic"]
                tmp_str += "<br>"
                tmp_str += verse["English"]
                tmp_str += "<br><br>"
            add_flashcard(
                front = f"{i}:{j+1}-{lim}",
                back = tmp_str,
                deck_name = deck
            )

In [ ]:
toAnki(100, 115, deck = "Quran")

# Query OpenAI with Arabic text - generate tabular meanings and open chat

In [ ]:
from Surah import Surah
Surah().getSurah()

In [ ]:
class OpenAI:
    """
    Query Arabic verse to ChatGPT
    Get list of all roots.
    Then send another query asking for all conjugations of roots
    """
    def __init__(self):
        pass